In [ ]:
import os
import math
from skimage.util import img_as_ubyte

In [ ]:
def calculate_psnr(img1, img2, border=0):
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    h, w = img1.shape[:2]
    img1 = img1[border:h - border, border:w - border]
    img2 = img2[border:h - border, border:w - border]

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


def calculate_ssim(img1, img2, border=0):

    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    h, w = img1.shape[:2]
    img1 = img1[border:h - border, border:w - border]
    img2 = img2[border:h - border, border:w - border]

    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1[:, :, i], img2[:, :, i]))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')


def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def apply_clahe(image_path, clip_limit=2.0, tile_grid_size=(8, 8)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Error: Image not found!")
        return None
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    clahe_img = clahe.apply(img)

    return clahe_img

def apply_clahe_color(image_path, clip_limit=2.0, tile_grid_size=(8, 8)):
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found!")
        return None
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_img)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)
    lab_img = cv2.merge((l, a, b))
    clahe_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2BGR)

    return clahe_img

def apply_clahe_hsi(image_path, clip_limit=2.0, tile_grid_size=(8, 8)):
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found!")
        return None
    hsi_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsi_img)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    v = clahe.apply(v)
    hsi_img = cv2.merge((h, s, v))
    clahe_img = cv2.cvtColor(hsi_img, cv2.COLOR_HSV2RGB)
    return clahe_img

def display_images(input_image, output_image):

    # Display the stacked images
    plt.figure(figsize=(10, 5))
    plt.imshow(input_image)
    plt.title('Original Image')
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.imshow(output_image)
    plt.title('CLAHE Image')
    plt.axis('off')
    plt.show()

def main(image_path,target_path,output_path,psnr_values,ssim_values):
    # clahe_img = apply_clahe(image_path)
    clahe_img = apply_clahe_hsi(image_path)

    if clahe_img is not None:
        original_img = cv2.imread(image_path)
        original_img=cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

        target_img = cv2.imread(target_path)
        target_img=cv2.cvtColor(target_img, cv2.COLOR_BGR2RGB)
        # display_images(original_img, clahe_img)
        psnr_values.append(calculate_psnr(target_img, clahe_img))
        ssim_values.append(calculate_ssim(img_as_ubyte(target_img), img_as_ubyte(clahe_img)))

        # output_image_path = '/content/drive/MyDrive/CLAHE_Output/clahe_output.png'
        cv2.imwrite(output_path, cv2.cvtColor(clahe_img, cv2.COLOR_BGR2RGB))
        # print(f"Output image saved as {output_image_path}")

# Run the main function with your image file path
if __name__ == "__main__":
    # input_image_path = '1.png'
    # input_directory='/content/drive/MyDrive/Retinexformer-master/data/NTIRE/mini_val/input/'
    input_directory='/content/drive/MyDrive/Retinexformer-master/basicsr/models/archs/LLFormer_input/'
    target_directory='/content/drive/MyDrive/Retinexformer-master/basicsr/models/archs/LLFormer_target/'
    # target_directory='/content/drive/MyDrive/Retinexformer-master/data/NTIRE/mini_val/target/'
    output_directory='/content/drive/MyDrive/CLAHE_Output/LLFormer_output'
    psnr_values = []
    ssim_values = []
    for filename in os.listdir(input_directory):
        if filename.endswith(('.png')):
            image_path = os.path.join(input_directory, filename)
            target_path = os.path.join(target_directory, filename)
            output_path = os.path.join(output_directory, filename)
            main(image_path,target_path,output_path,psnr_values,ssim_values)
    psnr_values = np.mean(np.array(psnr_values))
    ssim_values = np.mean(np.array(ssim_values))
    print("PSNR: %f " % (psnr_values))
    print("SSIM: %f " % (ssim_values))

PSNR: 9.530158 
SSIM: 0.374065 


In [ ]:
!ls

1.png  clahe_output.png  drive	sample_data
